In [8]:
import os
import cohere
import openai
import requests
from config import config
import yfinance as yf

In [9]:
COHERE_API_KEY = config["COHERE_API_KEY"]
PERPLEXITY_API_KEY = config["PERPLEXITY_API_KEY"]
FINANCIAL_MODELING_PREP_API_KEY = config["FINANCIAL_MODELING_PREP_API_KEY"]
SAMBA_API_KEY = config["SAMBA_API"]
OPENAI_API_KEY = config["OPENAI_API_KEY"]

co = cohere.Client(COHERE_API_KEY)
perclient = openai.OpenAI(api_key=PERPLEXITY_API_KEY, base_url="https://api.perplexity.ai")
sambaclient = openai.OpenAI(api_key=SAMBA_API_KEY, base_url="https://api.sambanova.ai/v1")
openclient = openai.OpenAI(api_key=OPENAI_API_KEY)

In [14]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf

def get_stock_price(symbol):
    """
    Fetch the current stock price and other data for the given symbol using Yahoo Finance.
    """
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        
        return {
            "symbol": symbol.upper(),
            "price": info.get('currentPrice', 'N/A'),
            "volume": info.get('volume', 'N/A'),
            "priceAvg50": info.get('fiftyDayAverage', 'N/A'),
            "priceAvg200": info.get('twoHundredDayAverage', 'N/A'),
            "EPS": info.get('trailingEps', 'N/A'),
            "PE": info.get('trailingPE', 'N/A'),
            "earningsAnnouncement": info.get('earningsTimestamp', 'N/A')
        }
    except Exception as e:
        return {"error": f"Could not fetch price for symbol: {symbol}. Error: {str(e)}"}

def get_company_financials(symbol):
    """
    Fetch basic financial information for the given company symbol using Yahoo Finance.
    """
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        
        return {
            "symbol": symbol.upper(),
            "companyName": info.get('longName', 'N/A'),
            "marketCap": info.get('marketCap', 'N/A'),
            "industry": info.get('industry', 'N/A'),
            "sector": info.get('sector', 'N/A'),
            "website": info.get('website', 'N/A'),
            "beta": info.get('beta', 'N/A'),
            "price": info.get('currentPrice', 'N/A')
        }
    except Exception as e:
        return {"error": f"Could not fetch financials for symbol: {symbol}. Error: {str(e)}"}

def get_income_statement(symbol):
    """
    Fetch last income statement for the given company symbol using Yahoo Finance.
    """
    try:
        stock = yf.Ticker(symbol)
        financials = stock.financials.iloc[:, 0]  # Get the most recent year
        
        return {
            "date": financials.name.strftime('%Y-%m-%d'),
            "revenue": financials.get('Total Revenue', 'N/A'),
            "gross profit": financials.get('Gross Profit', 'N/A'),
            "net Income": financials.get('Net Income', 'N/A'),
            "ebitda": financials.get('EBITDA', 'N/A'),
            "EPS": stock.info.get('trailingEps', 'N/A'),
            "EPS diluted": stock.info.get('trailingEps', 'N/A')  # Yahoo Finance doesn't provide separate diluted EPS
        }
    except Exception as e:
        return {"error": f"Could not fetch income statement for symbol: {symbol}. Error: {str(e)}"}


In [15]:
# Define the function schemas for LLM
functions = [
    {
        "name": "get_stock_price",
        "description": "Get the current stock price for a given company symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g., AAPL for Apple."
                }
            },
            "required": ["symbol"],
        },
    },
    {
        "name": "get_company_financials",
        "description": "Get basic financial information for a given company.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g., GOOGL for Alphabet Inc."
                }
            },
            "required": ["symbol"],
        },
    },
    {
        "name": "get_income_statement",
        "description": "Get the last income statement for a given company symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g., AMZN for Amazon."
                }
            },
            "required": ["symbol"],
        },
    },
]

In [18]:
# Chat Function:
MODEL_NAME = "llama-3.1-sonar-large-128k-online"

def chat_with_LLM(model_name = MODEL_NAME, client = perclient):
    messages = []
    print("I'm your Financial Analyst Assistant! Ask me anything about stock prices, company financials, or income statement.")
    while True:
        user_input = input("\nYou: ")
        print("User input:", user_input)
        if user_input.lower() in ["exit", "quit"]:
            print("Assistant: The discussion is terminated!")
            break

        messages.append({"role": "user", "content": user_input})

        response = client.chat.completions.create(
            model=model_name,
            messages=messages,
            functions=functions,
            function_call="auto",  # Let the model decide if a function needs to be called
        )

        response_message = response.choices[0].message

        # Check if the assistant wants to call a function
        if response_message.function_call:
            function_name = response_message.function_call.name
            arguments = response_message.function_call.arguments

            # Parse the arguments
            import json
            args = json.loads(arguments)

            # Call the appropriate function
            if function_name == "get_stock_price":
                function_response = get_stock_price(**args)
            elif function_name == "get_company_financials":
                function_response = get_company_financials(**args)
            elif function_name == "get_income_statement":
                function_response = get_income_statement(**args)
            else:
                function_response = {"error": "Function not found."}

            # Add the function's response to the conversation
            messages.append(response_message)  # Assistant's function call
            messages.append({
                "role": "function",
                "name": function_name,
                "content": json.dumps(function_response),
            })

            # Get the assistant's final response
            second_response = client.chat.completions.create(
                model=model_name,
                messages=messages,
            )

            print("FUNCTION CALL")
            final_message = second_response.choices[0].message.content
            print(f"Assistant: {final_message}")
            messages.append({"role": "assistant", "content": final_message})
            print(second_response)
            print(second_response.choices[0].message)
        else:
            # If no function call is needed, just output the assistant's message
            print("NO FUNCTION CALL")
            assistant_reply = response_message.content
            print(f"Assistant: {assistant_reply}")
            print(response.citations)
            messages.append({"role": "assistant", "content": assistant_reply})

# Run the chat function
if __name__ == "__main__":
    chat_with_LLM()

I'm your Financial Analyst Assistant! Ask me anything about stock prices, company financials, or income statement.
User input: latest nvidia revenue
NO FUNCTION CALL
Assistant: The latest revenue figures for NVIDIA are as follows:

## Annual Revenue for 2024:
NVIDIA reported an annual revenue of **$60.922 billion** for the fiscal year 2024, which represents a 125.85% increase from the $26.974 billion in 2023[2][4][5].

## Quarterly Revenue for the Second Quarter of 2024:
For the second quarter ended July 28, 2024, NVIDIA reported revenue of **$30.0 billion**, up 15% from the previous quarter and up 122% from the same quarter a year ago[1][2].

## Twelve Months Ending July 31, 2024:
The total revenue for the twelve months ending July 31, 2024, was **$96.307 billion**, marking a 194.69% increase year-over-year[2].
['https://investor.nvidia.com/financial-info/financial-reports/default.aspx', 'https://www.macrotrends.net/stocks/charts/NVDA/nvidia/revenue', 'https://nvidianews.nvidia.com/ne